In [1]:
import pandas as pd
import numpy as np
import pickle
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

### 데이터 합치기 - 뉴스, 블로그, 리뷰(플레이스토어 + 11번가)

#### [ 뉴스 ]

In [30]:
clova_news_df = pd.read_csv("../../data/naver_news/ktgigagenie_speaker_news.csv", index_col=0)
clova_news_df['source'] = 'news'
clova_news_df.head()

,title,date,publisher,url,text,source
0,"인공지능에 푹 빠진 이통사들… SKT, AI 스피커 ‘누구’ 공개, KT도 연내 출시",NaN,이투데이,http://www.etoday.co.kr/news/section/newsview....,NaN,news
1,"사투리까지 알아듣는다? SK텔레콤, ‘인공지능 홈 비서 ‘누구’ 출시",2016.08.31.,동아일보,https://news.naver.com/main/read.nhn?mode=LSD&...,[동아일보]SK텔레콤이 사투리까지 알아듣고 집안 내 가전기기 등을 제어할 수 있는 ...,news
2,스카이라이프 가입자도 AI 스피커 쓴다…대상 제품은 '구글 홈 미니',NaN,IT조선,http://it.chosun.com/site/data/html_dir/2019/0...,NaN,news
3,AI스피커 제조사 음질 경쟁…왜?,2019.05.12.,노컷뉴스언론사 선정,https://news.naver.com/main/read.nhn?mode=LSD&...,AI스피커 이용서비스 1위가 음악 감상[CBS노컷뉴스 김수영 기자]AI스피커 누구네...,news
4,"이통 3사, AI 스피커 경쟁 2라운드…이번엔 디스플레이",2019.04.29.,ZDNet Korea언론사 선정,https://news.naver.com/main/read.nhn?mode=LSD&...,"KT ‘기가지니 테이블TV’·SKT ‘누구 네모’ 출시…LGU+, 5월 초 신제품 ...",news


<br/><br/>
#### [ 블로그 ]

In [31]:
clova_blog_df = pd.read_csv("../../data/naver_blog/gigagenie_speaker_blog.csv", encoding='utf-16', index_col=0)
clova_blog_df['source'] = 'blog'
clova_blog_df.head()

,title,publish_datetime,text,url,source
0,내년엔 수많은 AI 비서들이 줄 섭니다…네이버·KT·LGU+에 삼성전자까지 가세,2016-12-16 15:25:00,NaN,http://blog.naver.com/PostView.nhn?blogId=sbki...,blog
1,"""사물을 연결하라""...그 다음은? [이코노믹리뷰]",2016-12-16 16:23:00,NaN,http://blog.naver.com/PostView.nhn?blogId=hone...,blog
2,"2017, 'AI비서'들이 거실로 몰려온다/국내 이통3사, 일제히 경쟁 돌입…IPT...",2016-12-19 02:10:00,NaN,http://blog.naver.com/PostView.nhn?blogId=hico...,blog
3,지금 ICT업계 핫이슈는 음석인식 기술,2016-12-19 16:56:00,NaN,http://blog.naver.com/PostView.nhn?blogId=just...,blog
4,인공지능 스피커,2016-12-22 09:58:00,NaN,http://blog.naver.com/PostView.nhn?blogId=hmmo...,blog


<br/>
#### [ 플레이스토어 리뷰 ]

In [32]:
clova_review_df = pd.read_csv('../../data/review/playstore_review_kt_gigagenie.csv')
clova_review_df['source'] = 'review'
clova_review_df.head()

,name,ratings,date,helpful vote,comment,source
0,황원욱,2,2019년 5월 12일,0,좋아하는 노래만 듣고싶은데 플레이리스트를 따로 설정해놓고 노래를 들을수 있는 방법은...,review
1,이름없음,2,2019년 5월 10일,2,기가지니 블루투스연결은 안되나요? 블루투스 연결하려하면 기기설정을 확인하라고뜨고 연...,review
2,경기도보안관,5,2019년 5월 10일,0,한가지 물어볼사항이 있는데요 kt간편로그인에 등록된 아이디가 있는데 제가 kt간편로...,review
3,이영희,3,2019년 5월 9일,0,에어컨 티비 켤때만 편한듭 솔직히 잘 못알아들어요. 그리고 문안인사 설정취소했는데도...,review
4,Sherliss Y,2,2019년 5월 12일,0,업데이트후 앱 우측하단에 음성명령을 텍스트로 써서 하는 기능이 사라진거같은데 왜 없...,review


<br/>
#### [ 11번가 리뷰 ]

In [33]:
_11st_clova_review_df = pd.read_csv('../../data/review/review_data_gigagenie.csv', index_col=0)
_11st_clova_review_df['source'] = 'review'
_11st_clova_review_df.head()

,index,nugu,score,source
0,1,배송이랑 등이랑 다잘왓구요 ㅎ 전등은 놓을데가 없어서 아직 못쓰고 잇네요 ㅎ 음악듣...,5,review
1,2,기가지니버디 성능 완정 말짱 꽝! 할 줄 아는게 없다! 클로바 적극 추천. 기가지니...,2,review
2,3,쇼핑비교사이트에서 비교하고 제일 저렴하게 구매했습니다.잘받았습니다.기본 10자 이상...,5,review
3,4,배송 빠르고 제품 아담하고 좋습니다.출력이 낮은 제품이기에 음량은 다소 약한 느낌이...,4,review
4,5,음질은 아주 좋다고 하긴 어렵지만 다른 음성인식 스피커들 보단 좀 괜찮은거 같아요....,5,review


<br/>
#### [ 데이터 합치기 ]


In [34]:
clova_df = pd.DataFrame({"date": pd.concat([clova_news_df.date, clova_blog_df.publish_datetime, 
                                            clova_review_df.date, pd.Series([None] * len(_11st_clova_review_df))]),
                         "text":pd.concat([clova_news_df.text, clova_blog_df.text, 
                                            clova_review_df.comment, _11st_clova_review_df.nugu]),
                         'source':pd.concat([clova_news_df.source, clova_blog_df.source, 
                                             clova_review_df.source, _11st_clova_review_df.source]),
                         'url':pd.concat([clova_news_df.url, clova_blog_df.url, 
                                          pd.Series([None] * len(clova_review_df)), pd.Series([None] * len(_11st_clova_review_df))])
                        })

In [35]:
clova_df = clova_df[clova_df.text.notnull()].copy()
clova_df.reset_index(inplace=True, drop=True)
clova_df

,date,text,source,url
0,2016.08.31.,[동아일보]SK텔레콤이 사투리까지 알아듣고 집안 내 가전기기 등을 제어할 수 있는 ...,news,https://news.naver.com/main/read.nhn?mode=LSD&...
1,2019.05.12.,AI스피커 이용서비스 1위가 음악 감상[CBS노컷뉴스 김수영 기자]AI스피커 누구네...,news,https://news.naver.com/main/read.nhn?mode=LSD&...
2,2019.04.29.,"KT ‘기가지니 테이블TV’·SKT ‘누구 네모’ 출시…LGU+, 5월 초 신제품 ...",news,https://news.naver.com/main/read.nhn?mode=LSD&...
3,2019.06.05.,음성인식 AI스피커가 디스플레이를 장착해 시각 콘텐츠도 제공하는 방향으로 진화하고 ...,news,https://news.naver.com/main/read.nhn?mode=LSD&...
4,2019.05.01.,그동안 AI 스피커는 음성을 주 인터페이스로 사용해왔다. 다양한 공간에서 손쉽게 쓸...,news,https://news.naver.com/main/read.nhn?mode=LSD&...
5,2019.06.02.,인공지능(AI) 스피커 시장이 미국과 중국 IT업체의 2강구도가 될 조짐을 보이고 ...,news,https://news.naver.com/main/read.nhn?mode=LSD&...
6,2019.04.29.,기가지니 테이블TV 공개…내 목소리 동화로 차별화[아이뉴스24 김문기 기자] 오직 ...,news,https://news.naver.com/main/read.nhn?mode=LSD&...
7,2019.05.06.,보이는 AI스피커 잇따라 출시모양도 크기도 비슷비슷차별점 찾기 힘들어AI 생태계 확...,news,https://news.naver.com/main/read.nhn?mode=LSD&...
8,2019.04.26.,아마존과 구글이 이끌어…국내에선 통신사 주도(지디넷코리아=권혜미 기자)스마트폰 다음...,news,https://news.naver.com/main/read.nhn?mode=LSD&...
9,2016.09.01.,[동아일보]‘스마트홈’ 서비스-가전 2題《 집이 똑똑해지고 있다. 이동통신회사는 ‘...,news,https://news.naver.com/main/read.nhn?mode=LSD&...


<br/><br/>
### 텍스트 전처리

In [36]:
def clear_content(temp):
    #want_remove = ['flash 오류를 우회하기 위한 함수 추가', 'function _flash_removeCallback', 
    #               '\n', '\.\.', '[\w]+ 기자', '\xa0', '\r', '\t']
    content = re.sub('[a-zA-Z0-9]+[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+', ' ', temp) # 이메일주소 없애기
    content = re.sub('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w_\.-]*)*\/?', ' ', content) # URL 없애기
    #content = re.sub('|'.join(want_remove), ' ', content)
    content = re.sub('\n|\r|\xa0', ' ', content)
    ㅠcontent = re.sub('[\(]', ' ', content) # 커머스(전자상거래)와' -> '커머스전자상거래와' 이렇게 변하던것을 -> '커머스 전자상거래'
    content = re.sub('~', ' ', content)
    #content = re.sub('[^\.\?\!\^^\♥\♡\w\d\s]','', content) # 불필요한 기호 없애기
    content = re.sub('\d', '', content)
    content = re.sub('[^\.\?\!\^^\♥\♡\w\s]','', content) # 불필요한 기호 없애기
    while '  ' in content: content = content.replace('  ', ' ') # 빈칸 없애주기 
    content = content.strip()
    return content

<br/><br/>
### 모델을 이용해서 네이버 블로그 데이터(中 clova) sentence별 긍정/부정 나누기

In [37]:
clova_df["preprocessed_text"] = clova_df['text'].map(clear_content) 

In [38]:
clova_df.head()

,date,text,source,url,preprocessed_text
0,2016.08.31.,[동아일보]SK텔레콤이 사투리까지 알아듣고 집안 내 가전기기 등을 제어할 수 있는 ...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,동아일보SK텔레콤이 사투리까지 알아듣고 집안 내 가전기기 등을 제어할 수 있는 누구...
1,2019.05.12.,AI스피커 이용서비스 1위가 음악 감상[CBS노컷뉴스 김수영 기자]AI스피커 누구네...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,AI스피커 이용서비스 위가 음악 감상CBS노컷뉴스 김수영 기자AI스피커 누구네모왼쪽...
2,2019.04.29.,"KT ‘기가지니 테이블TV’·SKT ‘누구 네모’ 출시…LGU+, 5월 초 신제품 ...",news,https://news.naver.com/main/read.nhn?mode=LSD&...,KT 기가지니 테이블TVSKT 누구 네모 출시LGU 월 초 신제품 공개 예정지디넷코...
3,2019.06.05.,음성인식 AI스피커가 디스플레이를 장착해 시각 콘텐츠도 제공하는 방향으로 진화하고 ...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,음성인식 AI스피커가 디스플레이를 장착해 시각 콘텐츠도 제공하는 방향으로 진화하고 ...
4,2019.05.01.,그동안 AI 스피커는 음성을 주 인터페이스로 사용해왔다. 다양한 공간에서 손쉽게 쓸...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,그동안 AI 스피커는 음성을 주 인터페이스로 사용해왔다. 다양한 공간에서 손쉽게 쓸...


In [39]:
del_idx = []
for num, i in enumerate(clova_df['preprocessed_text']):
    if i == '' or i == None or i == ' ':
        del_idx.append(num)
print(del_idx)

[]


In [40]:
clova_df.loc[clova_df.index[del_idx]]

,date,text,source,url,preprocessed_text


In [41]:
final_clova_df = clova_df.drop(clova_df.index[del_idx]).copy()

In [42]:
# 빈 데이터들을 다 삭제하고 난 뒤, 인덱스를 다시 0부터 순서대로 부여해 줌
final_clova_df.reset_index(drop=True, inplace=True)

In [43]:
final_clova_df.head()

,date,text,source,url,preprocessed_text
0,2016.08.31.,[동아일보]SK텔레콤이 사투리까지 알아듣고 집안 내 가전기기 등을 제어할 수 있는 ...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,동아일보SK텔레콤이 사투리까지 알아듣고 집안 내 가전기기 등을 제어할 수 있는 누구...
1,2019.05.12.,AI스피커 이용서비스 1위가 음악 감상[CBS노컷뉴스 김수영 기자]AI스피커 누구네...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,AI스피커 이용서비스 위가 음악 감상CBS노컷뉴스 김수영 기자AI스피커 누구네모왼쪽...
2,2019.04.29.,"KT ‘기가지니 테이블TV’·SKT ‘누구 네모’ 출시…LGU+, 5월 초 신제품 ...",news,https://news.naver.com/main/read.nhn?mode=LSD&...,KT 기가지니 테이블TVSKT 누구 네모 출시LGU 월 초 신제품 공개 예정지디넷코...
3,2019.06.05.,음성인식 AI스피커가 디스플레이를 장착해 시각 콘텐츠도 제공하는 방향으로 진화하고 ...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,음성인식 AI스피커가 디스플레이를 장착해 시각 콘텐츠도 제공하는 방향으로 진화하고 ...
4,2019.05.01.,그동안 AI 스피커는 음성을 주 인터페이스로 사용해왔다. 다양한 공간에서 손쉽게 쓸...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,그동안 AI 스피커는 음성을 주 인터페이스로 사용해왔다. 다양한 공간에서 손쉽게 쓸...


- 각 문헌을 sentence별로 나누기

In [44]:
import nltk.tokenize
final_clova_df['sentence_text'] = final_clova_df['preprocessed_text'].map(nltk.tokenize.sent_tokenize)

In [45]:
final_clova_df.head()

,date,text,source,url,preprocessed_text,sentence_text
0,2016.08.31.,[동아일보]SK텔레콤이 사투리까지 알아듣고 집안 내 가전기기 등을 제어할 수 있는 ...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,동아일보SK텔레콤이 사투리까지 알아듣고 집안 내 가전기기 등을 제어할 수 있는 누구...,[동아일보SK텔레콤이 사투리까지 알아듣고 집안 내 가전기기 등을 제어할 수 있는 누...
1,2019.05.12.,AI스피커 이용서비스 1위가 음악 감상[CBS노컷뉴스 김수영 기자]AI스피커 누구네...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,AI스피커 이용서비스 위가 음악 감상CBS노컷뉴스 김수영 기자AI스피커 누구네모왼쪽...,[AI스피커 이용서비스 위가 음악 감상CBS노컷뉴스 김수영 기자AI스피커 누구네모왼...
2,2019.04.29.,"KT ‘기가지니 테이블TV’·SKT ‘누구 네모’ 출시…LGU+, 5월 초 신제품 ...",news,https://news.naver.com/main/read.nhn?mode=LSD&...,KT 기가지니 테이블TVSKT 누구 네모 출시LGU 월 초 신제품 공개 예정지디넷코...,[KT 기가지니 테이블TVSKT 누구 네모 출시LGU 월 초 신제품 공개 예정지디넷...
3,2019.06.05.,음성인식 AI스피커가 디스플레이를 장착해 시각 콘텐츠도 제공하는 방향으로 진화하고 ...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,음성인식 AI스피커가 디스플레이를 장착해 시각 콘텐츠도 제공하는 방향으로 진화하고 ...,[음성인식 AI스피커가 디스플레이를 장착해 시각 콘텐츠도 제공하는 방향으로 진화하고...
4,2019.05.01.,그동안 AI 스피커는 음성을 주 인터페이스로 사용해왔다. 다양한 공간에서 손쉽게 쓸...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,그동안 AI 스피커는 음성을 주 인터페이스로 사용해왔다. 다양한 공간에서 손쉽게 쓸...,"[그동안 AI 스피커는 음성을 주 인터페이스로 사용해왔다., 다양한 공간에서 손쉽게..."


- 나눈 sentence들을 각각 형태소 분석하기

In [18]:
# 미등록단어 지정하여 Komoran 형태소 분석 
from konlpy.tag import Komoran

#dicpath = './unregistered_dic_blog_test.txt'
#komoran = Komoran(userdic= dicpath)
dicpath = './unregistered.txt'
komoran = Komoran(userdic=dicpath)

In [46]:
final_clova_df['pos_sentence_text'] = final_clova_df['sentence_text'].map(lambda l: [komoran.pos(sentence) for sentence in l])

In [47]:
#[" ".join([wors[0] for word in sent]) for sent in final_df['pos_sentence_text']]
def pos_by_sentence(l):
    result = []
    for sent in l:
        result.append(" ".join([word[0] for word in sent]))
    return result

final_clova_df['pos_sentence_text'] = final_clova_df['pos_sentence_text'].map(pos_by_sentence)

In [48]:
final_clova_df.head()

,date,text,source,url,preprocessed_text,sentence_text,pos_sentence_text
0,2016.08.31.,[동아일보]SK텔레콤이 사투리까지 알아듣고 집안 내 가전기기 등을 제어할 수 있는 ...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,동아일보SK텔레콤이 사투리까지 알아듣고 집안 내 가전기기 등을 제어할 수 있는 누구...,[동아일보SK텔레콤이 사투리까지 알아듣고 집안 내 가전기기 등을 제어할 수 있는 누...,[동아일보 SK텔레콤 이 사투리 까지 알아듣 고 집안 내 가전 기기 등 을 제어 하...
1,2019.05.12.,AI스피커 이용서비스 1위가 음악 감상[CBS노컷뉴스 김수영 기자]AI스피커 누구네...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,AI스피커 이용서비스 위가 음악 감상CBS노컷뉴스 김수영 기자AI스피커 누구네모왼쪽...,[AI스피커 이용서비스 위가 음악 감상CBS노컷뉴스 김수영 기자AI스피커 누구네모왼...,[AI 스피커 이용 서비스 위 가 음악 감상 CBS 노컷뉴스 김수영 기자 AI 스피...
2,2019.04.29.,"KT ‘기가지니 테이블TV’·SKT ‘누구 네모’ 출시…LGU+, 5월 초 신제품 ...",news,https://news.naver.com/main/read.nhn?mode=LSD&...,KT 기가지니 테이블TVSKT 누구 네모 출시LGU 월 초 신제품 공개 예정지디넷코...,[KT 기가지니 테이블TVSKT 누구 네모 출시LGU 월 초 신제품 공개 예정지디넷...,[KT 기가 지니 테이블 TVSKT 누구 네모 출시 LGU 월 초 신제품 공개 예정...
3,2019.06.05.,음성인식 AI스피커가 디스플레이를 장착해 시각 콘텐츠도 제공하는 방향으로 진화하고 ...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,음성인식 AI스피커가 디스플레이를 장착해 시각 콘텐츠도 제공하는 방향으로 진화하고 ...,[음성인식 AI스피커가 디스플레이를 장착해 시각 콘텐츠도 제공하는 방향으로 진화하고...,[음성 인식 AI 스피커 가 디스플레이 를 장착 하 아 시각 콘텐츠 도 제공 하 는...
4,2019.05.01.,그동안 AI 스피커는 음성을 주 인터페이스로 사용해왔다. 다양한 공간에서 손쉽게 쓸...,news,https://news.naver.com/main/read.nhn?mode=LSD&...,그동안 AI 스피커는 음성을 주 인터페이스로 사용해왔다. 다양한 공간에서 손쉽게 쓸...,"[그동안 AI 스피커는 음성을 주 인터페이스로 사용해왔다., 다양한 공간에서 손쉽게...","[그동안 AI 스피커 는 음성 을 주 인터페이스 로 사용 하 아 오 았 다 ., 다..."


#### sentence별 긍정(1) / 부정(-1)으로 나누기

In [49]:
def label_by_sentence(doc):
    loaded_vectorizer = pickle.load(open('./pretrained_vectorizer.pickle', 'rb'))
    loaded_model = pickle.load(open('./pretrained_model.pickle', 'rb'))
    
    tfidf_features = loaded_vectorizer.transform(doc)
    pred_labels = loaded_model.predict(tfidf_features)
    return pred_labels

final_clova_df['label_sentence_text'] = final_clova_df['pos_sentence_text'].map(label_by_sentence)

In [50]:
final_clova_df.to_csv("기가지니new.csv", encoding='utf-8')